In [5]:
from binascii import unhexlify,hexlify
from pydispatch import Dispatcher
from threading import Thread
from getComportclass import *
import paho.mqtt.client as mqtt
import time

# Classes

In [10]:
class readRBD7k(selecComport):
    def __init__(self):
        super().__init__(baudrate = 9600)
        self.command={
            "connect":"CC 80 03 03 01 01 00 00",
            "start" : "CC 80 03 03 01 02 00 03",
            "stop" : "CC 80 03 03 01 03 00 02",
            "lockscreen" : "CC 80 03 03 01 08 00 02",
            "unlockscreen" : "CC 80 03 03 01 08 00 02"
        }
        self.sys=0
        self.dia=0
        self.hr=0
        self.done=False
        self.readingData={
            "PRESSURE":{
                "SYS":self.sys,
                "DIA":self.dia},
            "HR":self.hr,
        }
        
    def HexStringtoByteArray(self,command):
        command=command.replace(" ","")
        command=unhexlify(command)
        return command
    
    def sendCommand(self,action):
        self.done=False
        if isinstance(self.ser, str) or self.ser.close():
            return "Devices not Connected"
        self.ser.write(self.HexStringtoByteArray(self.command[action.lower()]))
        print('Send Command: {0}'.format(str(self.command[action]).replace(" ","")))
        machineRes=self.machineRes()
        if machineRes == "AA80030301020003":
#             self.reading = Thread(target=self.readReady)
#             self.reading.start()
#             self.reading.join()
            return self.readingData
        else:
            return None
        
    def machineRes(self):
    #     Print out response of the machine [8 bytes]
        response = hexlify(self.ser.read(8)).decode('unicode-escape').upper()
        print("RBD-7000 Response: {0}".format(response))
        return response
    
    def readReady(self):
        line=[]
        startTime=time.perf_counter()
        finishTime=time.perf_counter()
        diff=finishTime-startTime
        timeout=30 #seconds
        while diff<timeout:
            if self.ser.in_waiting == 20:
                self.done=True
                for data in range(self.ser.in_waiting):
                    line.append(self.ser.read())
                break
            diff=finishTime-startTime
        return self.getsysdiahr(line)
    
    def getsysdiahr(self,hexstr):
        self.sys = int.from_bytes(hexstr[14],"big")
        self.dia = int.from_bytes(hexstr[16],"big")
        self.hr = int.from_bytes(hexstr[18],"big")
        return self.sys,self.dia,self.hr

# Test Functions

In [11]:
readSerial = readRBD7k()
readSerial.getSerObj()

'No ComPort Available'

In [12]:
readSerial.sendCommand('start')
time.sleep(5)
# print("hi")
readSerial.sendCommand('stop')
# readSerial.ser.close()

'Devices not Connected'